In [ ]:
# fst is the library I wrote for cosem, it's here on github:
# https://github.com/janelia-cosem/fst
from fst.io import read
from fst.pyramid import downscale
from fst.distributed import get_cluster
import dask.array as da
import numpy as np
from skimage.io import imsave
import os

# permissions error
# out_dir = '/groups/hess/hesslab/FIB-SEM_resolution/Atlas_data_sets/LID496_Atlas1/Segmentation/'
out_dir = '/groups/cosem/cosem/bennettd/for_gleb/'

# read data
data = read('/nrs/cosem/cosem/training/v0003.2/setup03/jrc_ctl-id8_a00/jrc_ctl-id8_a00_it650000.n5/ecs')
arr = da.from_array(data)

# define downsampling parameters
ds_factor = 10

# define downsampled dataset
arr_ds = downscale(arr, np.mean, (ds_factor,) * arr.ndim).astype(arr.dtype)

# get dask client that is connected to janelia cluster. This starts a 
# web server for monitoring progress.

cl = get_cluster()
print('See cluster status at', cl.dashboard_link)

# get 100 workers from janelia cluster
cl.cluster.scale(100)
# compute the downsampled data -- result is a numpy array
result = arr_ds.compute(scheduler=cl)
# retire the workers
cl.cluster.scale(0)

# save the data
imsave(os.path.join(out_dir, data.path + '.tif'), result)